# Philcomp Week 1 Tuesday

**Outline**

[Plan for course](#Plan-for-course)

[Primitive recursion idea and examples](#Primitive-recursion-idea-and-examples)

[Primitive recursion on strings](#Primitive-recursion-on-strings)

[Primitive recursion formal definition](#Primitive-recursion-formal-definition)

[The map](#The-map)

[Skolem on primitive recursive functions](#Skolem-on-primitive-recursive-functions)

[The limits of primitive recursion](#the-limits-of-primitive-recursion)

# Plan for course

Our plan for this course is to do an overview of the the philosophy of computation. 

We're going to starts with founding figures of Turing, Church, and Kleene and look at different conceptual analyses of computation (bounded and local search, beta reduction of lambda terms) which are related to the different models of computation (Turing machines, lambda calculi). 

Then we're going to look at effects of natural non-computability (Gödel's disjunction) and the non-computability of methods of verification (Rice's theorem). 

Then we're going to do topics such as:

- the apparent absoluteness of computation
- the confluence of different models of computation
- nature of feasibility (what is the intuitive notion)
- how semantics for procedures differs from semantics for indicative statements
- neural network models and its relation to computational theory of mind
- models of learning and its relation to reliability of inductive methods
- how computer learning is apparently different than human learning
- how fairness and explanation of machine learning methods can be assessed
- the physical implementation of computation and challenges from quantum computation.

This course does not presuppose any prior familiarity with computation or ability to program.

In terms of prior knowledge, I am going to structure this course so that it is to computation roughly what medical ethics is to medicine.

Everyone should be able to get a lot out of it, due in part to the ubiquity of computation in our everyday lives.

To help faciliate this, we are going to use the historical development to orientate the discussion.

Everything really begins in the 1930s with Turing and Church. 

But since this is week 1, we will ease into it by looking at precurors to computation in the 1920s.

# Primitive recursion idea and examples

**The idea** (of primitive recursion)

Define a function $f$ on natural numbers by specifying

*Base*: how it is defined on input $m=0$

*Iterator*: how it is defined on $m>0$ in terms of how it is defined on the previous step $m-1$.

To figure out the value of e.g. $f(3)$, write it in terms of $f(2)$ and the iterator; then write $f(2)$ in terms of $f(1)$ and the iterator; then write $f(1)$ in terms of $f(0)$ and the iterator; then use base to figure out $f(0)$.

---

**Example** (addition)

$$
n+m = \begin{cases}
    n & \text{if } m = 0 \\ 
   (n+(m-1))+1 & \text{if } m>0
\end{cases}
$$


To figure out the value of $5+2$, we proceed as follows:

$5+2 = (5+1)+1 =(5+0)+1+1 = 5+1+1 =6+1=7$

In [30]:
# here is a python implementation 

def add(n,m):
    if m ==0:
        return n
    return add(n,m-1)+1

add(5,2)

7

In [1]:
# for the next cell you might need to first load

# ! pip install metakernel

# just remove '#' from previous line and press shift + return

In [1]:
# this calls up python tutor (https://pythontutor.com), 
# which for very simple programs steps one through the computation

from metakernel import register_ipython_magics
register_ipython_magics()

In [2]:
%%tutor

# moral: modern implementations use the same 
# primitive recursive definition schema 
# and evaluation procedure

def add(n,m):
    if m ==0:
        return n
    return add(n,m-1)+1

add(5,2)

---

**Example** (multiplication)

$$
n*m = \begin{cases}
    0 & \text{if } m = 0 \\ 
    n*(m-1)+n & \text{if } m>0
\end{cases}
$$

In particular, we have:

$5*2 = 5*1+5 = 5*0+5+5=5+5=10$

In [33]:
# here is a python implementation, 
# where we use the built-in addition function

def mult(n,m):
    if m ==0:
        return 0
    return mult(n,m-1)+n

mult(5,2)

10

In [3]:
%%tutor

# note that it takes 14 steps

def mult(n,m):
    if m ==0:
        return 0
    return mult(n,m-1)+n

mult(5,2)


In [35]:
## below is a version where we appeal
## explicitly to our prior defn of addition
## instead of the built in version of addition
## note that it takes 63 steps.

In [36]:
%%tutor

def add(n,m):
    if m ==0:
        return n
    return add(n,m-1)+1

def mult(n,m):
    if m ==0:
        return 0
    return add(mult(n,m-1),n)

mult(5,2)

# Primitive recursion on strings

In many cases, it is more natural to compute on strings of symbols than on numbers. 

E.g. we could compute on strings from the latin alphabet $a, b, \ldots, z$.  

**The idea** (of primitive recursion)

Define a function $f$ on strings by specifying

*Base*: how it is defined on the the length zero string $\epsilon$

*Iterator*: how it is defined on length $m>0$ strings in terms of how it is defined on length $m-1$ strings and on the last entry.

To figure out the value of e.g. $f(aba)$, use iterator to compute it in terms of $f(ab)$ and last entry $a$; then use iterator to compute $f(ab)$ in terms of $f(a)$ and last entry $b$; then compute $f(\epsilon)$.

In this case, the iterator needs to tell you how to act when the last entry is $a$, how to act when it is $b$, etc. 

---

**Example** (flipping a's to b's)

Define 


$$
\mathrm{flip}(\sigma) = \begin{cases}
    \epsilon & \text{if $\sigma$ is length zero string $\epsilon$} \\ 
    \mathrm{flip}(\tau) b & \text{if  $\sigma = \tau a$} \\
    \mathrm{flip}(\tau) a & \text{if $\sigma = \tau b$}
\end{cases}
$$

In particular, where $\cdot^{\ast}$ changes $a$ to $b$ and vice-versa:

$\mathrm{flip}(aba) = \mathrm{flip}(ab)a^{\ast} = \mathrm{flip}(a)b^{\ast}a^{\ast} = a^{\ast} b^{\ast} a^{\ast} = b b^{\ast} a^{\ast}= baa^{\ast} =bab$

In [4]:
%%tutor

# a simple python program for flipping a's to b's
# and vice versa in a string of a's and b's.

def flip(str):
    if str =='':
        return ''
    if str[-1] == 'a':
        return flip(str[:-1])+'b'
    if str[-1] == 'b':
        return flip(str[:-1])+'a'    
flip('aba')

# Primitive recursion formal definition

Since it is in keeping with the historical record, we'll present the formal definition in terms of computation on numbers rather than strings. 

**Definition** (Definition by primitive recursion, binary functions)

Suppose a *base function* $B:\mathbb{N}\rightarrow \mathbb{N}$ and an *iterator function* $I:\mathbb{N}^{3}\rightarrow \mathbb{N}$ are given. Then the function $f:\mathbb{N}^{2}\rightarrow \mathbb{N}$ defined by *primitive recursion* from this base and iterator is the function which satisfies the following, for all $n,m$:

$$
f(n,m) = \begin{cases}
    B(n) & \text{if } m = 0 \\ 
   I(n,m-1,f(n,m-1)) & \text{if } m>0
\end{cases}
$$


One can similarly define unary, ternary functions by primitive recursion.

---

**Example** (Addition)

We must find functions $B$ and $I$ such that

$$
n+m = \begin{cases}
    B(n) & \text{if } m = 0 \\ 
   I(n,m-1,n+(m-1)) & \text{if } m>0
\end{cases}
$$


But just choose $B(n)=n$ and $I(n,m,\ell)=\ell+1$. 

Then we check:

$n+0 =n= B(n)$

$n+m = n+(m-1)+1 = I(n,m-1,n+(m-1))$.

---

**Example** (Multiplication)

We must find functions $B$ and $I$ such that

$$
n*m = \begin{cases}
    B(n) & \text{if } m = 0 \\ 
   I(n,m-1,n*(m-1)) & \text{if } m>0
\end{cases}
$$

But just choose $B(n)=0$ and $I(n,m,\ell)=\ell+n$. 

Then we check:

$n*0 =0= B(n)$

$n*m = n*(m-1)+n = I(n,m-1,n\ast(m-1))$.

---

**Definition** (Primitive Recursive Functions)


The *primitive recursive functions* on natural numbers are defined as follows:

- The identity function and constant functions and successor function $S(n)=n+1$ are primitive recursive. Further, the projection functions e.g. $f(n,m)=m$ and $g(n,m)=n$ are primitive recursive.

- The composition of primitive recursive functions is primitive recursive. That is, the result of putting primitive recursive functions into the inputs of other primitive recursive functions is primitive recursive. E.g. $f(x,y)=g(h(x,y), k(y))$ is primitive recursive if $g,h,k$ are primitive recursive.

- If the base function and the iterator function are primitive recursive, then the function defined from primitive recursion by this base and iterator is also primitive recursive.

- Nothing else is a primitive recursive function.

In other words: primitive recursive functions are closure of successor and projection and identity under composition and definition by primitive recursion. 

# The map

We're going to be looking at a number of different notions of computation in this course. 

There are two maps we will use to orientate ourselves.

The come from p. 104 of Soare 2016 and the cover of Immerman (see references).

This is the basic map:

![The map](map.png)

The map on left is of non-computable sets of numbers. The bottom is the computable or recursive sets of numbers.

The map on right zooms in on the computable sets of numbers and classifies them further. 

This is the map with primtive recursive sets highlighted:

![The map with pra highlighted](map-pra.png)

# Skolem on primitive recursive functions

The first to define the primitive recursive function was Skolem in the 1920s.

Skolem was motivated by considerations in the foundations of mathematics. 

He thought that set theory was not a good foundation for mathematics, for reasons including the following:

- There is no absolute notion of set-- it depends rather on what first-order model of set theory you are in (this is related to his epynmous paradox).

- It is circular to base number on set, since to prove things about sets you will need to prove things by mathematical induction, which presupposes numbers (this point, or a version of it, was originally made by Poincaré against the early Hilbert).

While set theory is widely used and applicable today, these two considerations of Skolem still resonate in modern philosophy of mathematics (see Button-Walsh 2016).

Our interest right now is in what Skolem thought the solution was. 

---

Skolem was moved to try to figure out how much mathematics could be done just with concepts related to number:

>If we consider the general theorems of arithmetic to be functional assertions and take the recursive mode of thought as a basis, then that science can be founded in a rigorous way without the use of [quantifiers] (Skolem 1967a p. 304)

>It occurred to me [in autumn of 1919] that already the use of logical variables that they call "real" would surely suffice to provide a foundation for a large part of mathematics (Skolem 1967b p. 332).

Since Skolem wanted to base mathematics off of recursion, he needed to *define recursion* and then show that a whole lot of mathematics was computable. 

And Skolem's definition of recursion was the notion of primitive recursive function. 

It is not clear if Skolem is presenting this as an axiomatization or as something else.

# The limits of primitive recursion

In 1925 (published in 1928), Ackermann found a function which was recursive in some sense and which was not primitive recursive. It is:

**Definition** (Ackermann function)

$$
\varphi_{\ell}(n,m) = \begin{cases}
    n+m & \text{if } \ell = 0 \\ 
   \alpha(n,\ell) & \text{if } m=0, \ell>0 \\
   \varphi_{\ell-1}(n,\varphi_{\ell}(n,m-1)) & \text{if } m>0, \ell>0
\end{cases}
$$

Where we define:

$$
\alpha(n,\ell) = \begin{cases}
    \ell & \text{if } \ell < 2 \\ 
    n & \text{if } \ell\geq 2
\end{cases}
$$

A function like this was found simulatenously by Sudan; see Claude et. al. 1979.

Experience suggests that this definition is hard to parse. 

---

The Ackermann function becomes more intuitive when one realizes that it is designed so that:

$\varphi_0(n,m)= n+m$

$\varphi_1(n,m)= n*m$

$\varphi_2(n,m) = n^m$

$\varphi_3(n,m) = n^{n^{\cdots^n}}$, with $m$-many $n$'s in the exponent.

Ackermann showed that it was not primitive recursive by showing that $\varphi(n,n,n)$ grew faster than any primitive recursive function. For a proof see Cori-Lascar 2001.

Unsurprisingly then, the Ackermann function is nearly impossible to compute on modern computers (see two cell/slides ahead).

---



At the time, the only public comment on the computability of this function was due to Ackerman's advisor Hilbert,who wrote that

> To be sure, we could now define $\varphi_{\ell}(n,m)$ for variable $\ell$ by means of substitutions and recursions, but these recursion would not be ordinary, stepwise ones; rather, we would be led to a manifold simulatenous recursion, that is a recursion on different variables at once, and a resolution of it into ordinary, stepwise recursions would be possible only if we make use of the notion of function variable (Hilbert 1967 p. 388; original from 1926).

From our perspective, what this shows is that the concept of recursion was in a bit of a flux in the 1920s. Primitive recursion was the paradigm they were working with, but they knew that there were functions which were recursive in some more general sense but which we were not primitive recursive.

In [ ]:
# here is a python implementation of the ackermann function
# it works for e.g. 3,2,2 but runs out of time/space for 3,3,3

import sys
sys.setrecursionlimit(10000000)

def alpha(n,l):
    if l <2:
        return l
    return n

def phi(l,n,m):
    if l==0:
        return n+m
    if m==0:
        return alpha(n,l) 
    return phi(l-1,n, phi(l,n,m-1)) 

phi(3,2,3)


---

The Ackermann function shows up occasionally in mathematical nature.

A subset $I$ of polynomials $\mathbb{Q}[X_1, \ldots, X_N]$ is an *ideal* if it is closed under addition, and by multiplication by other polynomials. 

Hilbert showed that these are ideals are always finitely generated (the *Hilbert Basis Theorem*, which also holds when the rationals are replaced by other fields). 

This implies that any ascending chain of ideals $I_0 \subseteq I_1 \subseteq I_s\subseteq\cdots$ eventually stops growing, e.g. there is $s$ such that $I_t=I_s$ for all $t\geq s$.

Further, if $I_s$ is generated by polynomials of degree $\leq s$, then all such chains stop at the same time, depending only on $N$.

The problem of computing from $N$ this stopping number is not primitive recursive. Indeed, it is an Ackermann-like function. See Socias 1992.

More generally, computational aspects of algebra geometry seem are good source of computable but hard to compute functions and operations. These flow in part from the Buchberger algorithm. See Cox et. al. 1997 and Simpson 1988

---

# References

Ackermann, W. (1928). Zum Hilbertschen Aufbau der reellen Zahlen. Mathematische Annalen, 99, 118–133. https://doi.org/10.1007/BF01459088

Button, T., & Walsh, S. (2016). Structure and Categoricity: Determinacy of Reference and Truth Value in the Philosophy of Mathematics. Philosophia Mathematica. Series III, 24(3), 283–307. http://dx.doi.org/10.1093/philmat/nkw007

Calude, C., Marcus, S., & Tevy, I. (1979). The first example of a recursive function which is not primitive recursive. Historia Mathematica, 6(4), 380–384. http://dx.doi.org/10.1016/0315-0860(79)90024-7

Cori, R., & Lascar, D. (2001). 5.1, 5.2.1 Primitive Recursive Functions and Sets; Ackermann’s function. In Mathematical Logic (Vol. 2, pp. 8–22). Oxford: Oxford University Press.

Cox, D. A., Little, J., & O’Shea, D. (1997). Ideals, Varieties, and Algorithms. Springer. https://link.springer.com/book/10.1007/978-3-319-16721-3

Immerman, N. (2012). Descriptive Complexity. Springer. https://people.cs.umass.edu/~immerman/book/descriptiveComplexity.html

Hilbert, D. (1967). On the Infinite. In J. Van Heijenoort (Ed.), From Frege to Gödel: A Source Book in Mathematical Logic, 1879-1931 (pp. 367–392). Harvard University Press. 

Originally published as: Hilbert, D. (1926). Über das Unendliche. Mathematische Annalen, 95(1), 161–190. https://doi.org/10.1007/BF01206605

Simpson, S. G. (1988). Ordinal numbers and the Hilbert basis theorem. Journal of Symbolic Logic, 53(3), 961–974. https://doi.org/10.2307/2274585 

Skolem, T. (1967a). The foundations of elementary arithmetic established by means of the recursive mode of thought, without the use of apparent variables ranging over infinite domains. In J. Van Heijenoort (Ed.), From Frege to Gödel: A Source Book in Mathematical Logic, 1879-1931 (pp. 302–333). Harvard University Press. 

Originally published as: Skolem, T. (1923a). Begründung der elementaren Arithmetik durch die rekurrierende Denkweise ohne Anwendung scheinbarer Veränderlichen mit unendlichem Ausdehnungsbereich. Videnskapsselskapets skrifter, I. Matematisk-naturvidenskabelig klasse, 6. Reprinted in: Skolem, T. (1970). Selected Works in Logic, By Th. Skolem. Edited by Jens Erik Fenstad. 

Skolem, T. (1967b). Some remarks on axiomatized set theory. In J. Van Heijenoort (Ed.), From Frege to Gödel: A Source Book in Mathematical Logic, 1879-1931 (pp. 390–301). Harvard University Press.

Originally published as: Skolem, T. (1923b). Einige Bemerkungen zur axiomatischen Begründung der Mengenlehre. In Matematikerkongressen i Helsingfors den 4-7 Juli 1922, Den femte skandinaviska matematikerskongressen, Redogörelse (pp. 217–232). Helsinki: Akademiska Bokhandeln. Repreinted in: Skolem, T. (1970). Selected Works in Logic, By Th. Skolem. Edited by Jens Erik Fenstad.

Socías, G. M. (1992). Length of Polynomial Ascending Chains and Primitive Recursiveness. Mathematica Scandinavica, 71(2), 181–205. https://www.jstor.org/stable/24492715